# Prompt

You are tasked with generating Python scripts that perform various BGP analysis tasks using the pybgpstream library. Please adhere to the following guidelines when writing the code:

- Main Loop Processing:
Do not use any filter attributes like stream.add_filter() or set filter parameters when initializing BGPStream.
All filtering and processing should occur within the main loop where you iterate over records and elements.

- Script Structure:
Start by importing necessary libraries, including pybgpstream and any others required for the task (e.g., datetime, collections).
Define a main function or functions that encapsulate the core logic.
Include a __main__ block or a usage example to demonstrate how to run the script.

- Key Processing Guidelines:
* Time Format: Define the time range as strings in the following format: from_time = "YYYY-MM-DD HH:MM:SS"
until_time = "YYYY-MM-DD HH:MM:SS"

* Stream Initialization: Use these time parameters during BGPStream initialization:
stream = pybgpstream.BGPStream(
    from_time=from_time,
    until_time=until_time,
    record_type="updates",
    collectors=collectors
)

* Iterating Over Records and Elements:
for rec in stream.records(): for elem in rec: # Processing logic goes here

* Accessing Element Attributes:
Timestamp: elem_time = datetime.utcfromtimestamp(elem.time)

Element Type (Announcement or Withdrawal): elem_type = elem.type # 'A' for announcements, 'W' for withdrawals

Fields Dictionary: fields = elem.fields

Prefix: prefix = fields.get("prefix") if prefix is None: continue # Skip if prefix is not present

AS Path: as_path_str = fields.get('as-path', "") as_path = as_path_str.split()

Peer ASN and Collector: peer_asn = elem.peer_asn collector = rec.collector

Communities: communities = fields.get('communities', [])

* Filtering Logic Within the Loop:
Filtering for a Specific ASN in AS Path: target_asn = '64500' if target_asn not in as_path: continue # Skip if target ASN is not in the AS path

Filtering for Specific Prefixes: target_prefixes = ['192.0.2.0/24', '198.51.100.0/24'] if prefix not in target_prefixes: continue # Skip if prefix is not in target prefixes

* Processing Key Values and Attributes:
Counting Announcements and Withdrawals: if elem_type == 'A': announcements[prefix] += 1 elif elem_type == 'W': withdrawals[prefix] += 1

Detecting AS Path Changes: if prefix in prefix_as_paths: if as_path != prefix_as_paths[prefix]: # AS path has changed prefix_as_paths[prefix] = as_path else: prefix_as_paths[prefix] = as_path

Analyzing Community Attributes: for community in communities: community_str = f"{community[0]}:{community[1]}" community_counts[community_str] += 1

Calculating Statistics (e.g., Average MED): med = fields.get('med') if med is not None: try: med_values.append(int(med)) except ValueError: pass # Ignore invalid MED values

* Detecting Hijacks: Compare the observed origin AS with the expected origin AS for target prefixes:
expected_origins = {'192.0.2.0/24': '64500', '198.51.100.0/24': '64501'}
if prefix in expected_origins:
    observed_origin = as_path[-1] if as_path else None
    expected_origin = expected_origins[prefix]
    if observed_origin != expected_origin:
        # Potential hijack detected
        print(f"Possible hijack detected for {prefix}: expected {expected_origin}, observed {observed_origin}")

* Detecting Outages:
* Monitor for withdrawals of prefixes without re-announcements:
Keep track of withdrawn prefixes and their timestamps
if elem_type == 'W':
    withdrawals[prefix] = elem_time
elif elem_type == 'A':
    # Remove from withdrawals if re-announced
    if prefix in withdrawals:
        del withdrawals[prefix]
Check if prefix remains withdrawn for a certain period (e.g., 30 minutes)
for prefix, withdrawal_time in list(withdrawals.items()):
    if elem_time - withdrawal_time > timedelta(minutes=30):
        # Outage detected for prefix
        print(f"Outage detected for {prefix} starting at {withdrawal_time}")
        del withdrawals[prefix]

* Detecting MOAS (Multiple Origin AS) Conflicts: Monitor prefixes announced by multiple origin ASNs
origin_asn = as_path[-1] if as_path else None
if origin_asn:
    if prefix not in prefix_origins:
        prefix_origins[prefix] = set()
    prefix_origins[prefix].add(origin_asn)
    if len(prefix_origins[prefix]) > 1:
        # MOAS conflict detected
        origins = ', '.join(prefix_origins[prefix])
        print(f"MOAS conflict for {prefix}: announced by ASNs {origins}")

* Analyzing AS Path Prepending: Detect AS path prepending by identifying consecutive repeated ASNs in the AS path:
last_asn = None
consecutive_count = 1
for asn in as_path:
    if asn == last_asn:
        consecutive_count += 1
    else:
        if consecutive_count > 1:
            prepending_counts[last_asn] += consecutive_count - 1
        consecutive_count = 1
    last_asn = asn
Check for prepending at the end of the path
if consecutive_count > 1 and last_asn:
    prepending_counts[last_asn] += consecutive_count - 1
    
* Handling IP Addresses and Prefixes:
Validating and Parsing IP Prefixes: import ipaddress try: network = ipaddress.ip_network(prefix) except ValueError: continue # Skip invalid prefixes

Here is your tasks:

###  list bgp messages for as18298 from rrc00 between 2024-10-28 and 2024-1029

In [5]:
import pybgpstream
from datetime import datetime
from collections import defaultdict
import ipaddress

def analyze_bgp_messages():
    # Define the time range
    from_time = "2024-10-28 12:00:00"
    until_time = "2024-10-28 12:05:00"

    # Specify the collector and target ASN
    collectors = ["rrc00"]
    target_asn = "3356"

    # Initialize BGPStream
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    # Initialize counters and data structures
    announcements = defaultdict(int)
    withdrawals = defaultdict(int)
    prefix_as_paths = {}
    community_counts = defaultdict(int)
    med_values = []

    # Iterate over records and elements
    for rec in stream.records():
        for elem in rec:
            # Process the element
            elem_time = datetime.utcfromtimestamp(elem.time)
            elem_type = elem.type  # 'A' for announcements, 'W' for withdrawals
            fields = elem.fields

            # Get the prefix
            prefix = fields.get("prefix")
            if prefix is None:
                continue  # Skip if prefix is not present

            # Get the AS path
            as_path_str = fields.get('as-path', "")
            as_path = as_path_str.split()

            # Check if the target ASN is in the AS path
            if target_asn not in as_path:
                continue  # Skip if target ASN is not in the AS path

            # Get peer ASN and collector
            peer_asn = elem.peer_asn
            collector = rec.collector

            # Count announcements and withdrawals
            if elem_type == 'A':
                announcements[prefix] += 1
            elif elem_type == 'W':
                withdrawals[prefix] += 1

            # Detect AS path changes
            if prefix in prefix_as_paths:
                if as_path != prefix_as_paths[prefix]:  # AS path has changed
                    prefix_as_paths[prefix] = as_path
            else:
                prefix_as_paths[prefix] = as_path

            # Analyze community attributes
            communities = fields.get('communities', [])
            for community in communities:
                community_str = f"{community[0]}:{community[1]}"
                community_counts[community_str] += 1

            # Calculate statistics (e.g., Average MED)
            med = fields.get('med')
            if med is not None:
                try:
                    med_values.append(int(med))
                except ValueError:
                    pass  # Ignore invalid MED values

    # Calculate average MED if values exist
    if med_values:
        med_avg = sum(med_values) / len(med_values)
        print(f"Average MED: {med_avg:.2f}")

    # Print the results
    print("Announcements:", dict(announcements))
    print("Withdrawals:", dict(withdrawals))
    print("Communities Count:", dict(community_counts))

if __name__ == "__main__":
    analyze_bgp_messages()

Announcements: {'103.177.86.0/24': 63, '103.177.87.0/24': 63, '216.190.56.0/24': 15, '2608:183:1::/48': 45, '38.146.198.0/23': 1704, '2a06:de01:88c::/48': 1, '2a02:ac80:7000::/48': 188, '2a02:ac80:b000::/48': 188, '2a02:ac80:400::/48': 188, '2a02:ac80:5000::/48': 188, '2a02:ac80:c0::/48': 188, '130.137.124.0/24': 16, '41.89.110.0/24': 33, '2a02:ac80::/29': 71, '2804:fdc::/32': 646, '177.184.88.0/24': 3, '169.145.140.0/23': 1783, '190.8.30.0/24': 59, '202.181.232.0/23': 22, '2a02:ac80:d00::/48': 257, '2a02:ac80:700::/48': 255, '2a02:ac80:100::/48': 257, '2a02:ac83::/32': 265, '2a02:ac80:60::/48': 185, '2a02:ac80:a0::/48': 195, '2a02:ac80:b0::/48': 186, '2a02:ac80:d000::/48': 186, '2a02:ac80:e00::/48': 185, '2a02:ac80:f00::/48': 185, '2a02:ac87:59cb::/48': 166, '177.91.59.0/24': 38, '27.97.124.0/24': 195, '27.97.127.0/24': 208, '27.97.122.0/24': 218, '27.97.126.0/24': 217, '27.97.125.0/24': 219, '27.97.123.0/24': 210, '27.97.120.0/24': 218, '27.97.121.0/24': 218, '80.211.126.0/24': 57, '

### detect hijacking from as3356 from oct 28 13pm to 13:30 pm 2024

In [1]:
import pybgpstream
from datetime import datetime, timedelta
import collections
import ipaddress

def detect_hijacking(from_time, until_time, target_asn):
    # Initialize BGPStream with the specified time range and record type
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates"
    )

    # Initialize dictionaries to track potential hijacks
    observed_origins = {}
    expected_origins = {'192.0.2.0/24': '3356', '198.51.100.0/24': '64501'}  # Example prefixes

    # Iterate over BGP records and their elements
    for rec in stream.records():
        for elem in rec:
            # Access element attributes
            elem_time = datetime.utcfromtimestamp(elem.time)
            elem_type = elem.type  # 'A' for announcements, 'W' for withdrawals
            fields = elem.fields

            # Get prefix and skip if not present
            prefix = fields.get("prefix")
            if prefix is None:
                continue

            # Get AS path and peer ASN
            as_path_str = fields.get('as-path', "")
            as_path = as_path_str.split()
            peer_asn = elem.peer_asn
            
            # Detect hijacks by comparing observed and expected origins
            if prefix in expected_origins:
                observed_origin = as_path[-1] if as_path else None
                expected_origin = expected_origins[prefix]
                
                if observed_origin != expected_origin:
                    print(f"Possible hijack detected for {prefix}: expected {expected_origin}, observed {observed_origin} from AS {peer_asn} at {elem_time}")

if __name__ == "__main__":
    # Define the time range for analysis
    from_time = "2024-10-28 13:00:00"
    until_time = "2024-10-28 13:30:00"
    target_asn = '3356'

    # Run the hijacking detection function
    detect_hijacking(from_time, until_time, target_asn)

In [9]:
gpt_response = """
ChatCompletionMessage(content='Here\'s a Python script that uses the `pybgpstream` library to list BGP messages for AS3356 from the collector `rrc00` between the specified date and time range of October 28, 2024, from 13:00 to 13:30. The script follows your guidelines closely and performs the necessary processing within the main loop.\n\n```python\nimport pybgpstream\nfrom datetime import datetime, timedelta\nfrom collections import defaultdict\n\ndef main():\n    # Time range for the BGPStream\n    from_time = "2024-10-28 13:00:00"\n    until_time = "2024-10-28 13:30:00"\n    \n    # Define the collector and target ASN\n    collectors = [\'rrc00\']\n    target_asn = \'3356\'\n    \n    # Initialize BGPStream\n    stream = pybgpstream.BGPStream(\n        from_time=from_time,\n        until_time=until_time,\n        record_type="updates",\n        collectors=collectors\n    )\n\n    # Initialize data structures\n    announcements = defaultdict(int)\n    withdrawals = defaultdict(int)\n\n    # Iterate over records and elements\n    for rec in stream.records():\n        for elem in rec:\n            elem_time = datetime.utcfromtimestamp(elem.time)\n            elem_type = elem.type  # \'A\' for announcements, \'W\' for withdrawals\n            fields = elem.fields\n            \n            # Get prefix and check for its presence\n            prefix = fields.get("prefix")\n            if prefix is None:\n                continue  # Skip if prefix is not present\n            \n            # Get AS path and filter for target ASN\n            as_path_str = fields.get(\'as-path\', "")\n            as_path = as_path_str.split()\n            if target_asn not in as_path:\n                continue  # Skip if target ASN is not in the AS path\n            \n            # Count announcements and withdrawals\n            if elem_type == \'A\':\n                announcements[prefix] += 1\n            elif elem_type == \'W\':\n                withdrawals[prefix] += 1\n            \n            # Print BGP message details\n            print(f"Time: {elem_time}, Type: {elem_type}, Prefix: {prefix}, AS Path: {\' -> \'.join(as_path)}, Collector: {rec.collector}")\n\n    # Print summary of announcements and withdrawals\n    print("\\nSummary:")\n    print("Announcements:")\n    for prefix, count in announcements.items():\n        print(f"{prefix}: {count}")\n\n    print("Withdrawals:")\n    for prefix, count in withdrawals.items():\n        print(f"{prefix}: {count}")\n\nif __name__ == "__main__":\n    main()\n```\n\n### Explanation of the Script:\n1. **Imports**: The script imports `pybgpstream`, `datetime`, and `defaultdict` from `collections` to manage the BGP records.\n  \n2. **Time Range**: The time range for analysis is defined as strings in the required format.\n\n3. **BGPStream Initialization**: The stream is initialized using the defined time range and the specified collector, `rrc00`.\n\n4. **Data Structures**: Two dictionaries are created to count announcements and withdrawals.\n\n5. **Main Loop**:\n   - The script iterates over each record in the stream and subsequently over each element in the record.\n   - It retrieves the timestamp, element type, and fields from the element.\n   - It checks if the prefix is present and skips processing if it\'s not.\n   - The AS path is extracted, and the script filters messages that do not include the target ASN (`3356`).\n   - Counts for announcements and withdrawals are updated accordingly.\n   - It prints the details of each BGP message in the specified format.\n\n6. **Summary Output**: At the end of processing, the script prints a summary of the counts of announcements and withdrawals for the prefixes observed.\n\n### Running the Script\nTo run this script, ensure you have the `pybgpstream` library installed in your Python environment, and execute the script in an environment that can access BGPStream data.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
"""

In [10]:
import re
def extract_code_from_reply(assistant_reply):
    code_pattern = r"```python\n(.*?)\n```"
    match = re.search(code_pattern, assistant_reply, re.DOTALL)
    if match:
        code = match.group(1)
        return code
    else:
        print("No code block found in the assistant's reply.")
        return None

In [11]:
if gpt_response:
    code = extract_code_from_reply(gpt_response)
    if code:
        # Save the code to a file
        with open('bgp_script.py', 'w') as f:
            f.write(code)
        print("Code saved to bgp_script.py")
    else:
        print("Failed to extract code from the assistant's reply.")

Code saved to bgp_script.py
